In [ ]:
import folium
from folium.plugins import GroupedLayerControl ###
from folium.plugins import TagFilterButton
from folium.plugins import FastMarkerCluster
from folium.plugins import MousePosition
from folium.plugins import Draw
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import rgb2hex, PowerNorm, Normalize, to_hex, TwoSlopeNorm
from matplotlib.patheffects import withSimplePatchShadow
import mplcursors
%matplotlib widget
import pandas as pd
import numpy as np
from shapely.geometry import *
from shapely.ops import transform
from altair import Chart
from inrix_data_science_utils.maps.quadkey import QuadKey
from datetime import datetime, timezone
from turfpy.measurement import boolean_point_in_polygon
from geojson import Point, Polygon, Feature
import random

In [ ]:
#### Import Latitude Longitude CSV
run_lat_lon = True
include_blur_in_exp_calc = True
if run_lat_lon == True :
    lat_lon_data = pd.read_csv('Blur_Olympia_7day_Lat_Lon_RD_2.csv') 
    lat_lon_sltot_wejo_ratio = (lat_lon_data['sl_real_count'].sum() + lat_lon_data['sl_blur_count'].sum()) / lat_lon_data['wejo_count'].sum()
    if include_blur_in_exp_calc == True :
        lat_lon_sltot_wejo_ratio = (lat_lon_data['sl_real_count'].sum() + lat_lon_data['sl_blur_count'].sum()) / lat_lon_data['wejo_count'].sum()
        lat_lon_data['sl_exp_count'] = lat_lon_data['wejo_count'] * lat_lon_sltot_wejo_ratio
    lat_lon_data['sl_vs_exp_count'] = round(lat_lon_data['sl_real_count'] - lat_lon_data['sl_exp_count'], 0)
    lat_lon_data['sl_vs_exp_percent'] = 100 * round(lat_lon_data['sl_vs_exp_count'] / lat_lon_data['sl_real_count'], 4)
else  :
    lat_lon_data = ""
lat_lon_data.head(2)

In [ ]:
#### Import QuadKey CSV and then add Lat/Lon details
run_quadkey = True
include_blur_in_exp_calc = True
#use_key_on_off
if run_quadkey == True : 
    quadkey_data = pd.read_csv('Blur_Olympia_7day_qk20.csv')

    if include_blur_in_exp_calc == True :
        quadkey_sltot_wejo_ratio = (quadkey_data['sl_real_count'].sum() + quadkey_data['sl_blur_count'].sum()) / quadkey_data['wejo_count'].sum()
        quadkey_data['sl_exp_count'] = quadkey_data['wejo_count'] * quadkey_sltot_wejo_ratio
    quadkey_data['sl_vs_exp_count'] = round(quadkey_data['sl_real_count'] - quadkey_data['sl_exp_count'], 0)
    quadkey_data['sl_vs_exp_percent'] = 100 * round(quadkey_data['sl_vs_exp_count'] / quadkey_data['sl_real_count'], 4)
    quadkey_data['qk_20'] = [QuadKey(str(i).zfill(20)) for i in quadkey_data['qk_20']]
    quadkey_data['lat_lon'] = quadkey_data['qk_20'].apply(QuadKey.to_geo)
    quadkey_data['lat_lon_bounds'] = quadkey_data['qk_20'].apply(QuadKey.get_geo_bounds)
    quadkey_data['mid_lat'] = [ (i[0][0] + i[1][0]) / 2 for i in quadkey_data['lat_lon_bounds']]
    quadkey_data['mid_lon'] = [ (i[0][1] + i[1][1]) / 2 for i in quadkey_data['lat_lon_bounds']]
    quadkey_data['mid_lat_2dp'] = [int(i * 100) / 100 for i in quadkey_data['mid_lat']]
    quadkey_data['mid_lon_2dp'] = [int(i * 100) / 100 for i in quadkey_data['mid_lon']]
    #quadkey_data['top_left_lat'], quadkey_data['top_left_lon'] = zip(*quadkey_data.lat_lon)
    lat_lon_data_join = lat_lon_data[["lat_2dp", "lon_2dp", "sl_blur_percent"]]
    lat_lon_data_join = lat_lon_data_join.rename(columns={'sl_blur_percent': 'sl_blur_percent_qk'})
    quadkey_data = quadkey_data.merge(lat_lon_data_join.reset_index(), how='left', 
                                          left_on=['mid_lat_2dp', 'mid_lon_2dp'], right_on=['lat_2dp', 'lon_2dp'])
else :
    quadkey_data = ""

#quadkey_data = quadkey_data[quadkey_data['sl_real_count'] ==]
quadkey_data#.head(2)
#list(quadkey_data)
#type(quadkey_data['lat_lon_bounds'][1])

In [ ]:
### Apply any pre-map filtering here 

if isinstance(lat_lon_data, pd.DataFrame) :
    lat_lon_data
    #lat_lon_data = lat_lon_data[lat_lon_data['sl_real_count'] == 0]

if isinstance(quadkey_data, pd.DataFrame) :
    quadkey_data
    #quadkey_data = quadkey_data[quadkey_data['sl_real_count'] == 0]

sltot_wejo_perc = 100 * ((quadkey_data['sl_real_count'].sum() + quadkey_data['sl_blur_count'].sum()) / quadkey_data['wejo_count'].sum())
sl_wejo_perc = 100 * (quadkey_data['sl_real_count'].sum() / quadkey_data['wejo_count'].sum())
print('Streetlight (inc blur) City Wide Data Count Change: {:+}'.format(round(sltot_wejo_perc - 100, 2)) + "%")
print('Streetlight (no blur) City Wide Data Count Change: {:+}'.format(round(sl_wejo_perc - 100, 2)) + "%")
print("Wejo Data Count: " + str(quadkey_data['wejo_count'].sum())) 
print("Streetlight Expected Data Count: " + str(round(quadkey_data['sl_exp_count'].sum(),0)))
print("Streetlight Actual Data Count: " + str(quadkey_data['sl_real_count'].sum()))
count_change = -1 * round(quadkey_data['sl_vs_exp_count'].sum(),0)
print('Streetlight Count vs Expected: {:+}'.format(count_change))
perc_change =  round(100 * ((quadkey_data['sl_real_count'].sum() / quadkey_data['sl_exp_count'].sum()) - 1),5)
print('Streetlight Count vs Expected Percentage Change: {:+}'.format(perc_change) + "%")


In [ ]:
def count_to_colour(variable, min_variable=0, mid_variable=5, max_variable=20, str_cmap='RdPu', scale='log'):
    """Transforms given value to 0-1 range and then finds corresponding hex colour.
    Choose the type of normalisation: linear or logarithmic, as well as range
    of scaled variable and matplotlib colourmap.
    """
    norm_func = (
        matplotlib.colors.PowerNorm(.5, vmin=min_variable, vmax=max_variable, clip=True) if scale == "log"
        #matplotlib.colors.TwoSlopeNorm( vmin=min_variable, vcenter=mid_variable, vmax=max_variable, clip=True) else if scale == "mid"
        else matplotlib.colors.Normalize(vmin=min_variable, vmax=max_variable, clip=True)
    )
    return to_hex(matplotlib.colormaps.get_cmap(str_cmap)(norm_func(variable)))

plot_quadkey_comp_ratio = True
plot_qk_colour = 'ratio'
plot_lat_lon_grid = 'crop'  # Line up lat lon grip with: 'round' or 'crop'
plot_lat_lon_2dp_circle = True
plot_ind_points = False

fmap = ""
fmap = folium.Map()

if plot_quadkey_comp_ratio == True :
    qk_comp_ratio_toggle = folium.FeatureGroup( name='Show QuadKey Ratios')
    data_qk_no_blank = quadkey_data[( quadkey_data['wejo_count'] != 0) | ( quadkey_data['sl_real_count'] != 0)]
    qk_list = data_qk_no_blank['qk_20'].tolist()
    sl_count_in_qks = data_qk_no_blank['sl_real_count'].tolist()
    wejo_count_in_qks = data_qk_no_blank['wejo_count'].tolist()
    ratio_in_qks = data_qk_no_blank['sl_wejo_ratio'].tolist()
    ind_list = data_qk_no_blank.index.tolist()
    sl_wejo_diff = data_qk_no_blank['sl_vs_exp_percent'].tolist()
    sl_exp_count = data_qk_no_blank['sl_exp_count'].tolist()
    sl_vs_exp_count = data_qk_no_blank['sl_vs_exp_count'].tolist()
    
    if plot_qk_colour == 'ratio' :
        color_offset_rule = matplotlib.colors.TwoSlopeNorm( vmin=0, vcenter=0.8, vmax=2)
        color_offset = color_offset_rule(sl_wejo_diff)
        weight_offset_rule = matplotlib.colors.Normalize( vmin=0, vmax=1000)
    else :
        matplotlib.colors.TwoSlopeNorm( vmin=-1000, vcenter=0, vmax=1000)
        color_offset = color_offset_rule(sl_vs_exp_count)
        weight_offset_rule = matplotlib.colors.Normalize( vmin=0, vmax=1000)

    for sl_count, wejo_count, qk, qk_ratio, sl_exp, qk_ratio_color, qk_count_diff in zip( sl_count_in_qks, wejo_count_in_qks, qk_list, sl_wejo_diff, sl_exp_count, color_offset, sl_vs_exp_count):
        # Get the geojson of our shape
        qk_geojson = qk.get_bounds_as_geojson()
        # Add a parameter with the colour we want
        if plot_qk_colour == 'ratio' :
            qk_geojson['colour'] = count_to_colour( qk_ratio_color, min_variable = 0, max_variable = 1, str_cmap='RdYlGn', scale='mid') #np.percentile(counts_in_qks, 99))
        else :
            qk_geojson['colour'] = count_to_colour( qk_ratio_color, min_variable = 0, max_variable = 1, str_cmap='RdYlGn', scale='mid') #np.percentile(counts_in_qks, 99))
        
        # Set the boundary weighting
        qk_count_diff_abs = abs( qk_count_diff)
        qk_geojson['weight'] = weight_offset_rule( qk_count_diff_abs)
        # Plot the geojson, with the style function pointing to our created feature
        folium.GeoJson(
            qk_geojson,
            # You can remove black borders by adding parameter 'opacity': 0
            style_function=lambda x: {'fillColor': x['colour'], 'color': 'black', 'weight': 0.2 + (x['weight']/2), 'opacity': 0.6, 'fillOpacity': 0.2 + (x['weight'] * 0.1)},
            tooltip = '<b>QuadKey:</b><br>' + str(qk) + '<br><b>Streetlight Count:</b><br>' + str(int(sl_count)) + '<br><b>Wejo Count:</b><br>'  
                        + str(int(wejo_count)) + '<br><b>Streetlight Expected:</b><br>' + str(round(sl_exp, 1)) + '<br><b>Streetlight Diff from Exp:</b><br>' + str(round(qk_count_diff, 0)) 
                        + '<br><b>Ratio vs Exp:</b><br>' + str(round(qk_ratio, 6)),
            legend_name="Streetlight Count vs Exp Ratio",
            highlight=True,
        ).add_to(qk_comp_ratio_toggle)
    fmap.add_child(qk_comp_ratio_toggle)

if plot_lat_lon_2dp_circle == True :
    lat_lon_2dp_circles = folium.FeatureGroup(name='Show Blurred Data Circles')
    data_to_plot_1 =  lat_lon_data #[lat_lon_data['sl_blur_count'] > 0 ]
    
    # We want to scale our drawn circle by the number of devices, so find the min and max values
    min_num_devices = min(data_to_plot_1['sl_blur_count'])
    max_num_devices = max(data_to_plot_1['sl_blur_count'])                    
    
    # Define how big/small we want our drawn circles
    smallest_circle_radius = 4
    biggest_circle_radius = 20
    
    # define a simple function which turns a number of devices into a radius
    get_radius = lambda x: ((x-min_num_devices)/(max_num_devices-min_num_devices)) * (biggest_circle_radius-smallest_circle_radius) + smallest_circle_radius
    
    # We want a red-green scale for smallest to biggest percentage increase
    
    min_perc = min(data_to_plot_1['sl_blur_percent'])
    max_perc = max(data_to_plot_1['sl_blur_percent'])
    
    # Add a circle markers to the map
    for i, row in data_to_plot_1.iterrows():
    
        popup_tab = pd.DataFrame(
                    data=[[row['lat_2dp'], row['lon_2dp'], str(row['sl_blur_count']), str(row['sl_real_count']), str(row['sl_blur_percent']) + "%", str(row['wejo_count']), str(round(row['sl_exp_count'],2)), str(row['sl_vs_exp_count'])]], 
                    columns = ["Latitude", "Longitude", "Blur Count", "Real Count", "Blur %", "Wejo Count", "Streetlight Expected Count", "Streetlight Expected +/-"])
        html = popup_tab.to_html(classes="table table-striped table-hover table-condensed table-responsive")
        popup = folium.Popup(html, max_width="100%")
    
        folium.CircleMarker(
            location = (row['lat_2dp'] + 0.01, row['lon_2dp'] - 0.005), # centre the circle
            radius = get_radius(row['sl_blur_count']), # Size of circle
            color = 'black', # Colour of the border
            #tags = [store, drop],
            weight = 1, # Thickness of border
            fill_color = count_to_colour(row['sl_blur_percent'], min_variable=0, max_variable=50, 
                                         str_cmap='RdYlGn_r', scale='log'), # Colour of the centre
            opacity = 0.6,
            fill_opacity = 0.4,
            #tooltip = (row['lat_2dp'], row['lon_2dp'], str(row['sl_blur_count']) , str(row['sl_real_count']), str(row['sl_blur_percent']) + "%"),
            tooltip = '<b>Latitude:</b><br>' + str(row['lat_2dp']) + '<br><b>Longitude:</b><br>' + str(row['lon_2dp']) + '<br><b>Streetlight Real Count:</b><br>'  
                        + str(row['sl_real_count']) + '<br><b>Streetlight Blur Data:</b><br>' + str(row['sl_blur_count']) + '<br><b>Streetlight Blur %:</b><br>' + str(round(row['sl_blur_percent'],2)) + "%" 
                        + '<br><b>Wejo Count:</b><br>' + str(row['wejo_count']),
            popup = popup,
        ).add_to(lat_lon_2dp_circles)
    fmap.add_child(lat_lon_2dp_circles)

if plot_lat_lon_grid == 'round' :

    data_to_plot_1 =  lat_lon_data
    min_lat = min(data_to_plot_1['lat_2dp'])
    max_lat = max(data_to_plot_1['lat_2dp'])
    min_lon = min(data_to_plot_1['lon_2dp'])
    max_lon = max(data_to_plot_1['lon_2dp'])
    lat_interval = 0.005
    lon_interval = 0.005
    grid = []
    
    for lat in np.arange(min_lat - lat_interval, max_lat + 2 * lat_interval, 2 * lat_interval):
        grid.append([[lat, min_lon - 4 * lon_interval],[lat, max_lon + 2 * lon_interval]])
    for lon in np.arange(min_lon - lon_interval, max_lon + 2 * lon_interval, 2 * lon_interval):
        grid.append([[min_lat - 2 * lat_interval, lon],[max_lat + 4 * lat_interval, lon]])
    for g in grid:
        folium.PolyLine(g, color="black", weight=1.5, opacity=0.5).add_to(fmap)

if plot_lat_lon_grid == 'crop' :
    data_to_plot_1 =  lat_lon_data
    min_lat = min(data_to_plot_1['lat_2dp'])
    max_lat = max(data_to_plot_1['lat_2dp'])
    min_lon = min(data_to_plot_1['lon_2dp'])
    max_lon = max(data_to_plot_1['lon_2dp'])
    lat_interval = 0.01
    lon_interval = 0.005
    grid = []
    
    for lat in np.arange(min_lat - 2 * lat_interval, max_lat + 4 * lat_interval, 2 * lat_interval):
        grid.append([[lat, min_lon - 4 * lon_interval],[lat, max_lon + 2 * lon_interval]])
    for lon in np.arange(min_lon - 4 * lon_interval, max_lon + 4 * lon_interval, 2 * lon_interval):
        grid.append([[min_lat - 2 * lat_interval, lon],[max_lat + 4 * lat_interval, lon]])
    for g in grid:
        folium.PolyLine(g, color="black", weight=1.5, opacity=0.5).add_to(fmap)

folium.plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(fmap)

MousePosition().add_to(fmap)

folium.LayerControl(collapsed=False).add_to(fmap)

Draw(export=True).add_to(fmap)

# Center the initial view on the drawn objects
fmap.fit_bounds(fmap.get_bounds())
fmap  

In [ ]:
fmap.save('Olympia_qk_&_Blur_Points_7day.html')

In [ ]:
lon_lat_capital_mall = [[[-122.939007,47.040007],[-122.931878,47.040007],[-122.926111,47.044711],[-122.926084,47.045735],[-122.929265,47.045951],[-122.934485,47.04598],[-122.93824,47.046945],[-122.94111,47.046974],[-122.941094,47.044547],[-122.93919,47.041509],[-122.939007,47.040007]]]
lon_lat_mall_target = [[[-122.937623,47.044774],[-122.936352,47.044774],[-122.936341,47.043531],[-122.937666,47.043549],[-122.937623,47.044774]]]
lon_lat_hospital = [[[-122.952965,47.041988],[-122.950026,47.041659],[-122.95009,47.045205],[-122.952826,47.045234],[-122.952965,47.041988]]]
lon_lat_mall_safeway = [[[-122.943277,47.046466],[-122.941411,47.046459],[-122.941405,47.045625],[-122.943347,47.045655],[-122.943277,47.046466]]]

#### Choose polygon here
lat_lon_choice = lon_lat_mall_target   #### Polygon choice
lon_lat_tuple = [[tuple(ele) for ele in sub] for sub in lat_lon_choice]
lon_lat_tuple_2 = lon_lat_tuple[0]

lon_lat_swap = [(sub[1], sub[0]) for sub in lon_lat_tuple_2]

polygon_bounds = Polygon([lon_lat_swap])

quadkey_polygon = quadkey_data
quadkey_polygon['in_polygon'] = [boolean_point_in_polygon([i, j], polygon_bounds) for i, j in zip(quadkey_polygon['mid_lat'], quadkey_polygon['mid_lon'])]

#quadkey_polygon = quadkey_polygon[quadkey_polygon['wejo_count'] == 403]
quadkey_polygon = quadkey_polygon[quadkey_polygon['in_polygon'] == True]

sl_wejo_perc = 100 * (quadkey_data['sl_real_count'].sum() / quadkey_data['wejo_count'].sum())
print('Streetlight City Wide Data Count Change: {:+}'.format(round(sl_wejo_perc - 100, 2)) + "%")
print("Wejo Data Count: " + str(quadkey_polygon['wejo_count'].sum())) 
print("Streetlight Expected Data Count: " + str(round(quadkey_polygon['sl_exp_count'].sum(),0)))
print("Streetlight Actual Data Count: " + str(quadkey_polygon['sl_real_count'].sum()))
count_change = round(quadkey_polygon['sl_vs_exp_count'].sum(),0)
print('Streetlight Count vs Expected: {:+}'.format(count_change))
perc_change =  round(100 * ((quadkey_polygon['sl_real_count'].sum() / quadkey_polygon['sl_exp_count'].sum()) - 1),3)
print('Streetlight Count vs Expected Percentage Change: {:+}'.format(perc_change) + "%")

In [ ]:
def count_to_colour(variable, min_variable=0, mid_variable=5, max_variable=20, str_cmap='RdPu', scale='log'):
    """Transforms given value to 0-1 range and then finds corresponding hex colour.
    Choose the type of normalisation: linear or logarithmic, as well as range
    of scaled variable and matplotlib colourmap.
    """
    norm_func = (
        matplotlib.colors.PowerNorm(.5, vmin=min_variable, vmax=max_variable, clip=True) if scale == "log"
        #matplotlib.colors.TwoSlopeNorm( vmin=min_variable, vcenter=mid_variable, vmax=max_variable, clip=True) else if scale == "mid"
        else matplotlib.colors.Normalize(vmin=min_variable, vmax=max_variable, clip=True)
    )
    #return matplotlib.colors.to_hex(cm.colormaps.get_cmap(str_cmap)(norm_func(variable)))
    return to_hex(matplotlib.colormaps.get_cmap(str_cmap)(norm_func(variable)))

plot_quadkey_comp_count = False
plot_quadkey_comp_ratio = True
plot_qk_colour = 'ratio'
plot_lat_lon_grid = 'crop'  # Line up lat lon grip with: 'round' or 'crop'
#plot_lat_lon_grid_crop = False
plot_lat_lon_2dp_circle = True
plot_ind_points = False

fmap_poly = ""
fmap_poly = folium.Map()

if plot_quadkey_comp_ratio == True :
    qk_comp_ratio_toggle = folium.FeatureGroup( name='Show QuadKey Ratios')
    data_qk_no_blank = quadkey_polygon[( quadkey_polygon['wejo_count'] != 0) | ( quadkey_polygon['sl_real_count'] != 0)]
    qk_list = data_qk_no_blank['qk_20'].tolist()
    sl_count_in_qks = data_qk_no_blank['sl_real_count'].tolist()
    wejo_count_in_qks = data_qk_no_blank['wejo_count'].tolist()
    ratio_in_qks = data_qk_no_blank['sl_wejo_ratio'].tolist()
    ind_list = data_qk_no_blank.index.tolist()
    sl_wejo_diff = data_qk_no_blank['sl_vs_exp_percent'].tolist()
    sl_exp_count = data_qk_no_blank['sl_exp_count'].tolist()
    sl_vs_exp_count = data_qk_no_blank['sl_vs_exp_count'].tolist()
    
    if plot_qk_colour == 'ratio' :
        color_offset_rule = matplotlib.colors.TwoSlopeNorm( vmin=0, vcenter=0.8, vmax=2)
        color_offset = color_offset_rule(sl_wejo_diff)
        weight_offset_rule = matplotlib.colors.Normalize( vmin=0, vmax=500)
    else :
        matplotlib.colors.TwoSlopeNorm( vmin=-100, vcenter=0, vmax=100)
        color_offset = color_offset_rule(sl_vs_exp_count)
        weight_offset_rule = matplotlib.colors.Normalize( vmin=0, vmax=100)
        
    #sl_vs_exp_count_abs = abs( sl_vs_exp_count)
    #weight_offset = weight_offset_rule( sl_vs_exp_count_abs)
    for sl_count, wejo_count, qk, qk_ratio, sl_exp, qk_ratio_color, qk_count_diff in zip( sl_count_in_qks, wejo_count_in_qks, qk_list, sl_wejo_diff, sl_exp_count, color_offset, sl_vs_exp_count):
        # Get the geojson of our shape
        qk_geojson = qk.get_bounds_as_geojson()
        # Add a parameter with the colour we want
        if plot_qk_colour == 'ratio' :
            qk_geojson['colour'] = count_to_colour( qk_ratio_color, min_variable = 0, max_variable = 1, str_cmap='RdYlGn', scale='mid') #np.percentile(counts_in_qks, 99))
        else :
            qk_geojson['colour'] = count_to_colour( qk_ratio_color, min_variable = 0, max_variable = 1, str_cmap='RdYlGn', scale='mid') #np.percentile(counts_in_qks, 99))
        
        # Set the boundary weighting
        qk_count_diff_abs = abs( qk_count_diff)
        qk_geojson['weight'] = weight_offset_rule( qk_count_diff_abs)
        # Plot the geojson, with the style function pointing to our created feature
        folium.GeoJson(
            qk_geojson,
            # You can remove black borders by adding parameter 'opacity': 0
            style_function=lambda x: {'fillColor': x['colour'], 'color': 'black', 'weight': 0.2 + (x['weight']/2), 'opacity': 0.6, 'fillOpacity': 0.2 + (x['weight'] * 0.1)},
            tooltip = '<b>QuadKey:</b><br>' + str(qk) + '<br><b>Streetlight Count:</b><br>' + str(int(sl_count)) + '<br><b>Wejo Count:</b><br>'  
                        + str(int(wejo_count)) + '<br><b>Streetlight Expected:</b><br>' + str(round(sl_exp, 1)) + '<br><b>Streetlight Diff from Exp:</b><br>' + str(round(qk_count_diff, 0)) 
                        + '<br><b>Ratio vs Exp:</b><br>' + str(round(qk_ratio, 6)),
            legend_name="Streetlight Count vs Exp Ratio",
            highlight=True,
        ).add_to(qk_comp_ratio_toggle)
    fmap_poly.add_child(qk_comp_ratio_toggle)

folium.plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(fmap_poly)

MousePosition().add_to(fmap_poly)

folium.LayerControl(collapsed=False).add_to(fmap_poly)

sl_wejo_perc = 100 * (quadkey_data['sl_real_count'].sum() / quadkey_data['wejo_count'].sum())
print('Streetlight City Wide Data Count Change: {:+}'.format(round(sl_wejo_perc - 100, 2)) + "%")
print("Wejo Data Count: " + str(quadkey_polygon['wejo_count'].sum())) 
print("Streetlight Expected Data Count: " + str(round(quadkey_polygon['sl_exp_count'].sum(),0)))
print("Streetlight Actual Data Count: " + str(quadkey_polygon['sl_real_count'].sum()))
count_change = round(quadkey_polygon['sl_vs_exp_count'].sum(),0)
print('Streetlight Count vs Expected: {:+}'.format(count_change))
perc_change =  round(100 * ((quadkey_polygon['sl_real_count'].sum() / quadkey_polygon['sl_exp_count'].sum()) - 1),3)
print('Streetlight Count vs Expected Percentage Change: {:+}'.format(perc_change) + "%")

# Center the initial view on the drawn objects
fmap_poly.fit_bounds(fmap_poly.get_bounds())
fmap_poly  

In [ ]:
point_data = pd.read_csv('Blur_Olympia_SL_1day_Data_Points.csv') 
point_data.head(2)

In [ ]:

point_data_filter = point_data[(point_data['blur'] == True)]
device_list = point_data_filter.deviceid.unique()
random_device = random.choice(device_list)
device_id = random_device

        #### Case 1: Blur across city to start, data jumps and then regular trip data but blur doesn't track right
##device_id = '201527036d990e967911df907c535288'
        #### Case 2: Only 3 data point. 400 second pause then jump across city
##device_id = '9f888311e4d9c5ef56b7b5f7e8b41432'
        #### Case 3: Blurring occuring South 2 grid rectangles
##device_id = 'e44494793cba1250efca37f9d4af1274'
        #### Case 4: Strong evidence of blurring occuring South 2 grid rectangles
##device_id = '58c63ae7112932fcac3141526a300c72'
        #### Case 5: No blurring but most data points are around a minute apart
##device_id = 'e3b2d56c89b8273c3e057c669c8e94ce'
        #### Case 6: 120 data points/6+ minutes of blurred data at the end of the trip
##device_id = '77da2da6018569bb19780615cebfe463'
        #### Case 7: 60 seconds between point 1 and 2, then a steady stream but later gaps start to appear. Big jump off the highway 
                    ### towards the end. Highway may be due to near edge of city (downloaded data range)
##device_id = '77da2da6018569bb19780615cebfe463'
        #### Case 8: Good example of several different occassions of blurring. Plus large gaps to start off the journey
##device_id = '05cdcf4b84433040a9bec408403b5350'
        #### Case 9: Jumps of up to 5 mins between point while driving across city
##device_id = '328b18e3d48a440dd656c210c210b942'
        #### Case 10: Single 'blurred' data point that isnt 2dp
##device_id = '18b9f1a889dcc08d3df743c8051c487c'
        #### Case 11: Several different 'blurred' data points that aren't 2dp
##device_id = '18b9f1a889dcc08d3df743c8051c487c'

device_data = point_data[point_data['deviceid'] == device_id]
device_data['date_time'] = pd.to_datetime(device_data['capturetsms'], unit = 'ms', origin='unix').dt.tz_localize('UTC' ).dt.tz_convert('America/Los_Angeles')
device_data['time_since'] = device_data['capturetsms'].diff() / 1000
device_data['lat_lon'] = device_data[['latitude', 'longitude']].values.tolist()
device_data['blur_next'] = device_data['blur'].shift(-1)
device_data['blur_prev'] = device_data['blur'].shift(1)

device_data.reset_index(drop=True)
device_data

In [ ]:
device_id


In [ ]:
#device_data.to_csv('device_data.csv', index=False)

In [ ]:
def count_to_colour(variable, min_variable=0, mid_variable=5, max_variable=20, str_cmap='RdPu', scale='log'):
    """Transforms given value to 0-1 range and then finds corresponding hex colour.
    Choose the type of normalisation: linear or logarithmic, as well as range
    of scaled variable and matplotlib colourmap.
    """
    norm_func = (
        matplotlib.colors.PowerNorm(.5, vmin=min_variable, vmax=max_variable, clip=True) if scale == "log"
        #matplotlib.colors.TwoSlopeNorm( vmin=min_variable, vcenter=mid_variable, vmax=max_variable, clip=True) else if scale == "mid"
        else matplotlib.colors.Normalize(vmin=min_variable, vmax=max_variable, clip=True)
    )
    return to_hex(matplotlib.colormaps.get_cmap(str_cmap)(norm_func(variable)))

plot_trip_line = True
plot_lat_lon_grid = 'crop'  # Line up lat lon grip with: 'round' or 'crop'
#plot_lat_lon_grid_crop = False
plot_lat_lon_2dp_circle = True
plot_ind_points = False

fmap_trip = ""
fmap_trip = folium.Map()

if plot_trip_line == True :
    trip_line = folium.PolyLine( device_data['lat_lon'].values.tolist(), weight=2, colour = "#8EE9FF").add_to(fmap_trip)
    attr = {"fill": "blue"}
    folium.plugins.PolyLineTextPath(trip_line, "\u25BA" + "         ", repeat=True, offset=0, attributes=attr).add_to(fmap_trip)
    
    trip_data = device_data.reset_index(drop=True)

    trip_circles = folium.FeatureGroup(name='Show Trip Data Circles')
    for i, row in trip_data.iterrows():
        if row['blur'] == True:
            fill_c = 'black'
        elif row['blur_next'] == True :
            fill_c = 'red'
        elif row['blur_prev'] == True :
            fill_c = 'red'
        elif row['time_since'] > 10 :
            fill_c = 'yellow'
        elif row['blur_next'] == True :
            fill_c = 'red'
        else :
            fill_c = 'green'
        
        folium.Circle(
            location = (row['latitude'], row['longitude']), # centre the circle
            radius = 40,
            color = 'black', # Colour of the border
            weight = 2, # Thickness of border
            fill_color = fill_c,
            opacity = 0.8,
            fill_opacity = 0.6,
            #tooltip = (row['latitude'], row['longitude'], str(row['sl_blur_count']) , str(row['sl_real_count']), str(row['sl_blur_percent']) + "%"),
            tooltip = '<b>Trip Status:</b><br>' + str(row['trip_status_1']) + '<br><b>Data Point:</b><br>' + str(i+1)
                    + '<br><b>Latitude:</b><br>' + str(row['latitude']) + '<br><b>Longitude:</b><br>' + str(row['longitude']) 
                    + '<br><b>Seconds Since Last Capture:</b><br>' + str(row['time_since']) + '<br><b>Date & Time:</b><br>' + str(row['date_time']),
        ).add_to(trip_circles)
    fmap_trip.add_child(trip_circles)

if plot_lat_lon_2dp_circle == True :
    lat_lon_2dp_circles = folium.FeatureGroup(name='Show Blurred Data Circles')
    
    data_to_plot_1 =  lat_lon_data #[lat_lon_data['sl_blur_count'] > 0 ]
    
    # We want to scale our drawn circle by the number of devices, so find the min and max values
    min_num_devices = min(data_to_plot_1['sl_blur_count'])
    max_num_devices = max(data_to_plot_1['sl_blur_count'])                    
    
    # Define how big/small we want our drawn circles
    smallest_circle_radius = 4
    biggest_circle_radius = 20
    
    # define a simple function which turns a number of devices into a radius
    get_radius = lambda x: ((x-min_num_devices)/(max_num_devices-min_num_devices)) * (biggest_circle_radius-smallest_circle_radius) + smallest_circle_radius
    
    # We want a red-green scale for smallest to biggest percentage increase
    min_perc = min(data_to_plot_1['sl_blur_percent'])
    max_perc = max(data_to_plot_1['sl_blur_percent'])
    
    # Add a circle markers to the map
    for i, row in data_to_plot_1.iterrows():
        popup_tab = pd.DataFrame(
                    data=[[row['lat_2dp'], row['lon_2dp'], str(row['sl_blur_count']), str(row['sl_real_count']), str(row['sl_blur_percent']) + "%", str(row['wejo_count']), str(round(row['sl_exp_count'],2)), str(row['sl_vs_exp_count'])]], 
                    columns = ["Latitude", "Longitude", "Blur Count", "Real Count", "Blur %", "Wejo Count", "Streetlight Expected Count", "Streetlight Expected +/-"])
        html = popup_tab.to_html(classes="table table-striped table-hover table-condensed table-responsive")
        popup = folium.Popup(html, max_width="100%")
    
        folium.CircleMarker(
            location = (row['lat_2dp'] + 0.01, row['lon_2dp'] - 0.005), # centre the circle
            radius = get_radius(row['sl_blur_count']), # Size of circle
            color = 'black', # Colour of the border
            #tags = [store, drop],
            weight = 1, # Thickness of border
            fill_color = count_to_colour(row['sl_blur_percent'], min_variable=15, max_variable=100, 
                                         str_cmap='RdYlGn_r', scale='log'), # Colour of the centre
            opacity = 0.6,
            fill_opacity = 0.4,
            #tooltip = (row['lat_2dp'], row['lon_2dp'], str(row['sl_blur_count']) , str(row['sl_real_count']), str(row['sl_blur_percent']) + "%"),
            tooltip = '<b>Latitude:</b><br>' + str(row['lat_2dp']) + '<br><b>Longitude:</b><br>' + str(row['lon_2dp']) + '<br><b>Streetlight Real Count:</b><br>'  
                        + str(row['sl_real_count']) + '<br><b>Streetlight Blur Data:</b><br>' + str(row['sl_blur_count']) + '<br><b>Streetlight Blur %:</b><br>' + str(round(row['sl_blur_percent'],2)) + "%" 
                        + '<br><b>Wejo Count:</b><br>' + str(row['wejo_count']),
            popup = popup,
        ).add_to(lat_lon_2dp_circles)
    fmap_trip.add_child(lat_lon_2dp_circles)
        
if plot_lat_lon_grid == 'crop' :
    data_to_plot_1 =  lat_lon_data
    min_lat = min(data_to_plot_1['lat_2dp'])
    max_lat = max(data_to_plot_1['lat_2dp'])
    min_lon = min(data_to_plot_1['lon_2dp'])
    max_lon = max(data_to_plot_1['lon_2dp'])
    lat_interval = 0.01
    lon_interval = 0.005
    grid = []
    
    for lat in np.arange(min_lat - 2 * lat_interval, max_lat + 4 * lat_interval, 2 * lat_interval):
        grid.append([[lat, min_lon - 4 * lon_interval],[lat, max_lon + 2 * lon_interval]])
    for lon in np.arange(min_lon - 4 * lon_interval, max_lon + 4 * lon_interval, 2 * lon_interval):
        grid.append([[min_lat - 2 * lat_interval, lon],[max_lat + 4 * lat_interval, lon]])
    for g in grid:
        folium.PolyLine(g, color="black", weight=1.5, opacity=0.5).add_to(fmap_trip)

folium.plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(fmap_trip)

MousePosition().add_to(fmap_trip)
folium.LayerControl(collapsed=False).add_to(fmap_trip)
Draw(export=True).add_to(fmap_trip)

# Center the initial view on the drawn objects
fmap_trip.fit_bounds(fmap_trip.get_bounds())
fmap_trip  

In [ ]:
lat_lon_data

In [ ]:
lat_2dp_group = lat_lon_data.groupby('lat_2dp')['sl_blur_count'].sum()
lat_2dp_group

In [ ]:
lon_2dp_group = lat_lon_data.groupby('lon_2dp')['sl_blur_count'].sum()
lon_2dp_group

In [ ]:
trip_count = len(point_data_filter.deviceid.unique())
key_on_count = len(point_data_filter[point_data_filter['trip_status_1'] == "KEYON"])
key_off_count = len(point_data_filter[point_data_filter['trip_status_1'] == "KEYOFF"])

print(str(trip_count) + " unique trips")
print(str(key_on_count) + " trip starts")
print(str(key_off_count) + " trip ends")

### Need to remove trips that start/end near boundary to get an actual ratio